In [1]:
# ! pip install catboost
# ! pip install seaborn
# ! pip install imbalanced-learn

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import random
from tqdm import tqdm as tqdm
import pickle


# -------------------------------------------------
import lightgbm as lgbm
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from catboost import Pool, CatBoostClassifier
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GroupShuffleSplit

# ---------------------------------------------------
from sklearn.cluster import KMeans

# from featexp import get_univariate_plots
from imblearn.under_sampling import ClusterCentroids
from imblearn.under_sampling import TomekLinks
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer

# -------------------------data-----------------------#
import requests
from urllib.parse import urlparse
from pathlib import Path
from datetime import datetime

# Required libraries
import tifffile as tiff
import datetime
import matplotlib.pyplot as plt

import rasterio
from rasterio.plot import show
from rasterio.windows import Window
from rasterio.enums import Resampling
from rasterio.transform import Affine

import seaborn as sns
import matplotlib.pyplot as plt

In [8]:
# Path to the input GeoTIFF file
input_file = r"D:\projects\kelp_wanted\data\train_satellite\AA498489_satellite.tif"
mask_file = r"D:\projects\kelp_wanted\data\train_kelp\AA498489_kelp.tif"

In [10]:
train_satellite_path = r"D:\projects\kelp_wanted\data\train_satellite"
train_label_path = r"D:\projects\kelp_wanted\data\train_kelp"

In [11]:
# Get the list of train files
train_input = os.listdir(train_satellite_path)
train_input_id = [d.split("_")[0] for d in train_input if d.endswith(".tif")]

# Get the list of train labels
train_label = os.listdir(train_label_path)

In [ ]:
train_input_id

In [13]:
# Define scale factor and offset
scale_factor = 0.0000275
offset = -0.2

In [8]:
# # make a dataframe with column for 7 bands, label and id
# df = pd.DataFrame(columns=["B01", "B02", "B03", "B04", "B05", "B06", "B07", "label", "id"])

In [9]:
# train_input_id[:10]

In [ ]:
dfs = []  # Initialize list to store DataFrames
# count = 0

# Loop over each file
for file in train_input_id:  # Adjust the range as needed
    # Read the GeoTIFF file
    with rasterio.open(train_satellite_path + "\\" + file + "_satellite.tif") as src:
        # Read all bands
        band_data = src.read().astype(np.float32)  # Convert to float32
        # Get the shape of the image
        height, width = src.height, src.width
        # Apply scale factor and offset to all bands except band 6 and band 7
        for i in range(7):
            if i != 5 and i != 6:  # Skip band 6 and band 7
                band_data[i] = band_data[i] * scale_factor + offset

        # Create a mask where band 7 values are greater than 0
        land_mask = band_data[6] > 0
        # Apply the land mask to all bands except band 6 and 7
        for i in range(5):
            band_data[i][land_mask] = 0

        # Find the indices of pixels not in the land mask
        non_land_mask_indices = np.where(~land_mask)

        # Read the binary mask TIFF file
        with rasterio.open(train_label_path + "\\" + file + "_kelp.tif") as mask_src:
            # Read the mask data
            kelp_mask = mask_src.read(1)
            # Find the locations where the mask values are 1
            kelp_mask_indices = np.where(kelp_mask == 1)

        # Select a random sample of pixels
        num_random_samples = len(kelp_mask_indices[0])

        random_indices = random.sample(
            range(len(non_land_mask_indices[0])), num_random_samples
        )
        # Combine indices of kelp mask pixels and random sample pixels
        selected_indices = (
            np.concatenate(
                (kelp_mask_indices[0], non_land_mask_indices[0][random_indices])
            ),
            np.concatenate(
                (kelp_mask_indices[1], non_land_mask_indices[1][random_indices])
            ),
        )

        # Filter pixels with band 6 values greater than 0
        selected_indices_filtered = (
            selected_indices[0][band_data[5][selected_indices] == 0],
            selected_indices[1][band_data[5][selected_indices] == 0],
        )

        # Extract band values for each selected pixel
        pixel_values = band_data[
            :, selected_indices_filtered[0], selected_indices_filtered[1]
        ].T

        # Create a DataFrame for the selected pixels
        df = pd.DataFrame(pixel_values, columns=[f"B0{i+1}" for i in range(7)])

        # Create an array of labels
        labels = np.zeros(len(selected_indices_filtered[0]))
        labels[: len(kelp_mask_indices[0])] = 1  # Set labels to 1 for kelp mask pixels
        df["label"] = labels
        df["id"] = file
        # count = count + 1
        # df["count"] = count

        # Append DataFrame to the list
        dfs.append(df)

# Concatenate all DataFrames in the list
final_df = pd.concat(dfs, ignore_index=True)

# Shuffle the DataFrame rows
df_shuffled = final_df.sample(frac=1, random_state=42).reset_index(drop=True)

In [38]:
len(np.where(df_shuffled.label == 0)[0])

4423238

In [ ]:
# # Read the GeoTIFF file
# with rasterio.open(input_file) as src:
#     # Read all bands
#     band_data = src.read().astype(np.float32)  # Convert to float32
#     # Get the shape of the image
#     height, width = src.height, src.width
#     # Apply scale factor and offset to all bands except band 6 and band 7
#     for i in range(7):
#         if i != 5 and i != 6:  # Skip band 6 and band 7
#             band_data[i] = band_data[i] * scale_factor + offset
            
#     # Create a mask where band 6 values are greater than 0
#     land_mask = band_data[6] > 0
#     # Apply the land mask to all bands except band 6
#     for i in range(7):
#         band_data[i][land_mask] = 0

#     # Find the indices of pixels not in the land mask
#     non_land_mask_indices = np.where(~land_mask)
    
#     # Read the binary mask TIFF file
#     with rasterio.open(mask_file) as mask_src:
#         # Read the mask data
#         kelp_mask = mask_src.read(1)
#         # Find the locations where the mask values are 1
#         kelp_mask_indices = np.where(kelp_mask == 1)

#     # Select a random sample of pixels
#     num_random_samples = min(
#         len(non_land_mask_indices[0]), len(kelp_mask_indices[0]), 1000
#     )
#     random_indices = random.sample(
#         range(len(non_land_mask_indices[0])), num_random_samples
#     )
#     # Combine indices of kelp mask pixels and random sample pixels
#     selected_indices = (
#         np.concatenate(
#             (kelp_mask_indices[0], non_land_mask_indices[0][random_indices])
#         ),
#         np.concatenate(
#             (kelp_mask_indices[1], non_land_mask_indices[1][random_indices])
#         ),
#     )

#     # Create a dictionary to store band values for each selected pixel
#     data_dict = {f"B0{i+1}": band_data[i][selected_indices] for i in range(7)}
#     # Create an array of labels
#     labels = np.zeros(len(selected_indices[0]))
#     labels[:len(kelp_mask_indices[0])] = 1  # Set labels to 1 for kelp mask pixels
    
#     # Add labels as a new column to the DataFrame
#     data_dict["label"] = labels
#     data_dict["id"] = input_id

#     # Create a DataFrame from the dictionary
#     df = pd.DataFrame(data_dict)
#     # Shuffle the DataFrame rows
#     df_shuffled = df.sample(frac=1, random_state=42).reset_index(drop=True)

In [40]:
# Calculate NDVI
# (N - R)/(N + R)	
df_shuffled["NDVI"] = (df_shuffled["B02"] - df_shuffled["B03"]) / (
    df_shuffled["B02"] + df_shuffled["B03"]
)

In [41]:
# Calculate NDWI
# (N - S1) / (N + S1)
df_shuffled["NDWI"] = (df_shuffled["B02"] - df_shuffled["B01"]) / (
    df_shuffled["B02"] + df_shuffled["B01"]
)

In [42]:
# Calculate G-NDVI
# (N - G) / (N + G)
df_shuffled["G-NDVI"] = (df_shuffled["B02"] - df_shuffled["B04"]) / (
    df_shuffled["B02"] + df_shuffled["B04"]
)

In [43]:
# Calculate GRVI 
# (N / G)
df_shuffled["GRVI"] = (df_shuffled["B02"] / df_shuffled["B04"])

In [44]:
# # TGI	Triangular Greenness Index	-(-0.5*((R-B)*(R-G)-(R-G)*(R-B)))
# df_shuffled["TGI"] = -(-0.5 * (
#     (df_shuffled["B03"] - df_shuffled["B05"]) * (df_shuffled["B03"] - df_shuffled["B04"]) 
#     - 
#     (df_shuffled["B03"] - df_shuffled["B04"]) * (df_shuffled["B03"] - df_shuffled["B05"])
#     )
# )

In [45]:
# TVI	Triangular Vegetation Index	0.5 * (120 * (N - G) - 200 * (R - G))
df_shuffled["TVI"] = 0.5 * (120 * (df_shuffled["B02"] - df_shuffled["B04"]) - 200 * (df_shuffled["B03"] - df_shuffled["B04"]))

In [46]:
# # Calculate NIRv
# # ((N - R) / (N + R)) * N
# df_shuffled["NIRv"] = ((df_shuffled["B02"] - df_shuffled["B03"]) / (df_shuffled["B02"] + df_shuffled["B03"])) * df_shuffled["B03"]

In [47]:
# # CIgreen(Chlorophyll Index Green)
# # (N / G) - 1.0
# df_shuffled["CIgreen"] = (df_shuffled["B02"] / df_shuffled["B04"])

In [ ]:
# # Find count of label 1 and 0
#     # label_counts = df_shuffled['label'].value_counts()


#     # Path to save the filtered GeoTIFF file
#     output_file = (
#         r"D:\projects\kelp_wanted\data\test_data\AA498489_satellite_filtered.tif"
#     )

#     # Calculate the new transformation matrix with a vertical flip
#     transform = Affine(
#         src.transform.a, 0, src.transform.c, 0, -src.transform.e, src.transform.f
#     )

#     # Write the filtered image to a new GeoTIFF file
#     with rasterio.open(
#         output_file,
#         "w",
#         driver="GTiff",
#         width=width,
#         height=height,
#         count=7,
#         dtype=band_data.dtype,
#         crs=src.crs,
#         transform=transform,
#     ) as dst:
#         for i in range(7):
#             dst.write(band_data[i], i + 1)

# print("Filtered image saved as:", output_file)

### feature importance after training
### Feature Importance: [ 7.9042736  24.65610458  8.17403439  4.31374823 10.15926229  7.50094776 11.60824219 10.62018195  6.10334926  8.95985576]

In [ ]:
df_shuffled.head(10)

In [51]:
df_shuffled.to_csv(r"D:\projects\kelp_wanted\data\csv\df_shuffled.csv", index=False)

In [7]:
df_shuffled = pd.read_csv(r"D:\projects\kelp_wanted\data\csv\df_shuffled.csv")

# Train/Test Split

In [8]:
groups = df_shuffled["id"]
unique_groups = df_shuffled["id"].unique()
len(groups)

9095439

#### 1

In [72]:
# # Split the dataset into features (X) and labels (y)
# X = df_shuffled.drop(columns=["id", "label"])  # Features excluding 'id' and 'label' columns
# y = df_shuffled["label"]  # Target variable

# # Specify groups based on the 'id' column
# groups = df_shuffled["id"]

# # Initialize the GroupShuffleSplit with a specified test size
# group_splitter = GroupShuffleSplit(test_size=0.3, random_state=42)

# # Split the data into train and test sets while preserving group membership
# train_idx, test_idx = next(group_splitter.split(X, y, groups=groups))

# # Split the data into train and test sets
# X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
# y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

#### 2

In [9]:
# Split the dataset into features (X) and labels (y)
X = df_shuffled.drop(
    columns=["id", "label", "B06", "B07"]
)  # Features excluding 'id' and 'label' columns
y = df_shuffled["label"]  # Target variable

# Specify groups based on the 'id' column
groups = df_shuffled["id"]

# Initialize the GroupShuffleSplit with a specified test size
group_splitter = GroupShuffleSplit(test_size=0.3, random_state=42)

# Split the data into train and test sets while preserving group membership
train_idx, test_idx = next(group_splitter.split(X, y, groups=groups))

# Split the data into train and test sets
X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

# Create CatBoost Pool objects for training and validation data
train_pool = Pool(data=X_train, label=y_train)
test_pool = Pool(data=X_test, label=y_test)

### Train the model

In [ ]:
# Initialize and train the CatBoostClassifier
model = CatBoostClassifier(
    task_type="GPU",
    iterations=5000,
    learning_rate=0.001,
    random_strength=0.1,
    depth=8,
    loss_function="Logloss",
)
model.fit(X_train.fillna(0), y_train)

## other methods

#### Validation Accuracy: 0.8712742664509043
#### Feature Importance: [ 7.9042736  24.65610458  8.17403439  4.31374823 10.15926229  7.50094776 11.60824219 10.62018195  6.10334926  8.95985576]

In [78]:
# Initialize and train the CatBoostClassifier
model = CatBoostClassifier(
    task_type="GPU",
    iterations=5000,
    learning_rate=0.001,
    random_strength=0.1,
    depth=8,
    loss_function="MultiClass",
    l2_leaf_reg=3,  # L2 regularization strength
    # subsample=0.8,  # Subsample ratio for training
    verbose=100,  # Print every 100 iterations
)
model.fit(
    train_pool,
    eval_set=test_pool,  # Use a validation set to monitor performance
    early_stopping_rounds=50,  # Stop training if the performance doesn't improve for 100 iterations
    plot=True,  # Plot the training metrics during training
)


# Get predictions on the validation set
test_preds = model.predict(X_test)

# Evaluate accuracy
val_accuracy = accuracy_score(y_test, test_preds)
print("Validation Accuracy:", val_accuracy)

# Get feature importance
feature_importance = model.get_feature_importance(train_pool)
print("Feature Importance:", feature_importance)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.6925527	test: 0.6925324	best: 0.6925324 (0)	total: 170ms	remaining: 14m 11s
100:	learn: 0.6446243	test: 0.6415225	best: 0.6415225 (100)	total: 5.98s	remaining: 4m 50s
200:	learn: 0.6047965	test: 0.5991532	best: 0.5991532 (200)	total: 12s	remaining: 4m 47s
300:	learn: 0.5714413	test: 0.5636174	best: 0.5636174 (300)	total: 17.8s	remaining: 4m 38s
400:	learn: 0.5432996	test: 0.5335149	best: 0.5335149 (400)	total: 23.1s	remaining: 4m 25s
500:	learn: 0.5194222	test: 0.5079496	best: 0.5079496 (500)	total: 28.6s	remaining: 4m 16s
600:	learn: 0.4990395	test: 0.4861508	best: 0.4861508 (600)	total: 33.9s	remaining: 4m 8s
700:	learn: 0.4816199	test: 0.4675319	best: 0.4675319 (700)	total: 39.3s	remaining: 4m
800:	learn: 0.4666162	test: 0.4515025	best: 0.4515025 (800)	total: 44.8s	remaining: 3m 54s
900:	learn: 0.4537052	test: 0.4377203	best: 0.4377203 (900)	total: 50.4s	remaining: 3m 49s
1000:	learn: 0.4425303	test: 0.4258202	best: 0.4258202 (1000)	total: 56.2s	remaining: 3m 44s
1100:	l

In [82]:
model.save_model(r"D:\projects\kelp_wanted\weights\new\catboost_model.bin")

#### XGBoost Validation Accuracy: 0.8732260125923625

In [80]:
from xgboost import XGBClassifier

# Initialize and train the XGBoost Classifier
xgb_model = XGBClassifier(
    n_estimators=5000,
    learning_rate=0.001,
    max_depth=8,
    reg_lambda=3,  # L2 regularization strength
    verbosity=1,  # Print messages during fitting
    device="cuda",  # Use GPU for training
)

xgb_model.fit(
    X_train,
    y_train,
    eval_set=[(X_test, y_test)],  # Use a validation set to monitor performance
    early_stopping_rounds=50,  # Stop training if the performance doesn't improve for 50 iterations
    verbose=True,  # Print messages during fitting
)

# Get predictions on the validation set
test_preds_xgb = xgb_model.predict(X_test)

# Evaluate accuracy
val_accuracy_xgb = accuracy_score(y_test, test_preds_xgb)
print("XGBoost Validation Accuracy:", val_accuracy_xgb)

c:\Users\HDSL77\anaconda3\envs\bd_maskrcnn\lib\site-packages\xgboost\sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-logloss:0.69234
[1]	validation_0-logloss:0.69175
[2]	validation_0-logloss:0.69116
[3]	validation_0-logloss:0.69058
[4]	validation_0-logloss:0.68999
[5]	validation_0-logloss:0.68941
[6]	validation_0-logloss:0.68882
[7]	validation_0-logloss:0.68824
[8]	validation_0-logloss:0.68766
[9]	validation_0-logloss:0.68708
[10]	validation_0-logloss:0.68650
[11]	validation_0-logloss:0.68592
[12]	validation_0-logloss:0.68535
[13]	validation_0-logloss:0.68477
[14]	validation_0-logloss:0.68420
[15]	validation_0-logloss:0.68362
[16]	validation_0-logloss:0.68305
[17]	validation_0-logloss:0.68248
[18]	validation_0-logloss:0.68191
[19]	validation_0-logloss:0.68134
[20]	validation_0-logloss:0.68077
[21]	validation_0-logloss:0.68021
[22]	validation_0-logloss:0.67964
[23]	validation_0-logloss:0.67908
[24]	validation_0-logloss:0.67852
[25]	validation_0-logloss:0.67795
[26]	validation_0-logloss:0.67739
[27]	validation_0-logloss:0.67683
[28]	validation_0-logloss:0.67627
[29]	validation_0-loglos

c:\Users\HDSL77\anaconda3\envs\bd_maskrcnn\lib\site-packages\xgboost\core.py:160: UserWarning: [01:14:15] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


XGBoost Validation Accuracy: 0.8732260125923625


In [81]:
xgb_model.save_model(r"D:\projects\kelp_wanted\weights\new\xgb_model.bin")

c:\Users\HDSL77\anaconda3\envs\bd_maskrcnn\lib\site-packages\xgboost\core.py:160: UserWarning: [01:14:58] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)


In [12]:
import lightgbm as lgb

# Initialize and train the LightGBM Classifier
lgb_model = lgb.LGBMClassifier(
    n_estimators=5000,
    learning_rate=0.001,
    max_depth=8,
    reg_lambda=3,  # L2 regularization strength
    verbosity=-1,  # Suppress warnings
    device="gpu",  # Use GPU for training
)

lgb_model.fit(
    X_train,
    y_train,
    eval_set=[(X_test, y_test)],  # Use a validation set to monitor performance
    # early_stopping_rounds=50,  # Stop training if the performance doesn't improve for 50 iterations
    # verbose=True,  # Print messages during fitting
)

# Get predictions on the validation set
test_preds_lgb = lgb_model.predict(X_test)

# Evaluate accuracy
val_accuracy_lgb = accuracy_score(y_test, test_preds_lgb)
print("LightGBM Validation Accuracy:", val_accuracy_lgb)

LightGBM Validation Accuracy: 0.8713271951598253


In [20]:
model_name = 'light_gbm.sav'
pickle.dump(
    lgb_model, open(os.path.join(r"D:\projects\kelp_wanted\weights\new", model_name), "wb")
)

In [ ]:
# check if X_train contians NaN
np.any(np.isnan(X_train))

# print those values
X_train[np.isnan(X_train)]

In [ ]:
X_train

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Initialize and train the Random Forest Classifier
rf_model = RandomForestClassifier(
    n_estimators=500,
    max_depth=8,
    min_samples_split=2,
    min_samples_leaf=1,
    random_state=42,
    n_jobs=-1,  # Use all available CPU cores
)

rf_model.fit(X_train, y_train)

# Get predictions on the validation set
test_preds_rf = rf_model.predict(X_test)

# Evaluate accuracy
val_accuracy_rf = accuracy_score(y_test, test_preds_rf)
print("Random Forest Validation Accuracy:", val_accuracy_rf)

### Save the Model

In [ ]:
# Define the file path where you want to save the model
model_file_path = r"D:\projects\kelp_wanted\model\catboost_model_2.bin"

In [ ]:
# Save the model
model.save_model(model_file_path)

### Load the Models

#### CatBoost

In [ ]:
# Load the saved model
loaded_model = CatBoostClassifier()
loaded_model.load_model(model_file_path)

In [5]:
# Load the model
xgb_model = xgb.Booster()
xgb_model.load_model(r"D:\projects\kelp_wanted\weights\new\xgb_model.bin")

### Test the model

In [ ]:
# Get predicted probabilities
preds = loaded_model.predict_proba(X_test.fillna(0))

### Generate result

In [ ]:
# Convert predicted probabilities to class predictions
threshold = 0.5  # Adjust threshold as needed
pred_labels = (preds[:, 1] > threshold).astype(int)

# Compute confusion matrix
conf_matrix = confusion_matrix(y_test, pred_labels)

# Compute total number of samples
total_samples = len(y_test)

# Convert confusion matrix to percentage values
conf_matrix_percentage = (conf_matrix / total_samples) * 100

print("Confusion Matrix (Percentage):")
print(conf_matrix_percentage)

Confusion Matrix (Percentage):
[[20.81728859  3.5355666 ]
 [ 4.02242749 71.62471733]]


In [ ]:
# Compute accuracy
accuracy = accuracy_score(y_test, pred_labels)

# Compute precision
precision = precision_score(y_test, pred_labels)

# Compute recall
recall = recall_score(y_test, pred_labels)

# Compute F1-score
f1 = f1_score(y_test, pred_labels)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

In [ ]:
# Generate confusion matrix
conf_matrix = confusion_matrix(y_test, pred_labels)

# Plot heatmap
plt.figure(figsize=(5, 5))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicted labels")
plt.ylabel("True labels")
plt.title("Confusion Matrix")
plt.show()

# TEST

In [ ]:
def calculate_bands(df):
    # Calculate NDVI
    # (N - R)/(N + R)
    df["NDVI"] = (df["B02"] - df["B03"]) / (
        df["B02"] + df["B03"]
    )
    # Calculate NDWI
    # (N - S1) / (N + S1)
    df["NDWI"] = (df["B02"] - df["B01"]) / (
        df["B02"] + df["B01"]
    )
    # Calculate G-NDVI
    # (N - G) / (N + G)
    df["G-NDVI"] = (df["B02"] - df["B04"]) / (
        df["B02"] + df["B04"]
    )
    # Calculate GRVI
    # (N / G)
    df["GRVI"] = df["B02"] / df["B04"]
    # # TGI	Triangular Greenness Index	-(-0.5*((R-B)*(R-G)-(R-G)*(R-B)))
    # df["TGI"] = -(-0.5 * (
    #     (df["B03"] - df["B05"]) * (df["B03"] - df["B04"])
    #     -
    #     (df["B03"] - df["B04"]) * (df["B03"] - df["B05"])
    #     )
    # )
    # TVI	Triangular Vegetation Index	0.5 * (120 * (N - G) - 200 * (R - G))
    df["TVI"] = 0.5 * (
        120 * (df["B02"] - df["B04"])
        - 200 * (df["B03"] - df["B04"])
    )
    
    return df

In [ ]:
test_file = r"D:\projects\kelp_wanted\data\train_satellite\JS569579_satellite.tif"

In [ ]:
test_satellite_path = r"D:\projects\kelp_wanted\data\test_satellite"

In [ ]:
# Get the list of train files
test_satellite = os.listdir(test_satellite_path)
test_satellite_id = [d.split("_")[0] for d in test_satellite if d.endswith(".tif")]

In [ ]:
test_satellite_id[:4]

['AA408972', 'AA804258', 'AB121622', 'AB144957']

In [ ]:
for file in tqdm(test_satellite_id):
# Read the id_kelp.tif file
    with rasterio.open(test_satellite_path + "\\" + file + "_satellite.tif") as kelp_src:
    # with rasterio.open(test_file) as kelp_src:
        kelp_mask = kelp_src.read(1)
        band_data = kelp_src.read().astype(np.float32)  # Convert to float32

    # Create a zero image of the same dimension
    zero_image = np.zeros_like(kelp_mask)

    # """
    # Read all bands
    # Get the shape of the image
    height, width = kelp_src.height, kelp_src.width
    # Apply scale factor and offset to all bands except band 6 and band 7
    for i in range(5):
        band_data[i] = band_data[i] * scale_factor + offset

    # Create a land mask
    land_mask = band_data[6] > 0  # Assuming band 6 represents land
    # """
    # Read non-land pixels from the input image
    non_land_pixels = band_data[:, ~land_mask]

    # Remember the positions of non-land pixels
    non_land_indices = np.where(~land_mask)

    # Extract band values for each selected pixel
    pixel_values = band_data[:, non_land_indices[0], non_land_indices[1]].T

    # Create a DataFrame for the selected pixels
    df = pd.DataFrame(pixel_values, columns=[f"B0{i+1}" for i in range(7)])

    # dfs.append(df)
    # # Concatenate all DataFrames in the list
    # test_final_df = pd.concat(df, ignore_index=True)
    # Shuffle the DataFrame rows
    # df = df.sample(frac=1, random_state=42).reset_index(drop=True)
    
    # Calculate NDVI
    # (N - R)/(N + R)
    df["NDVI"] = (df["B02"] - df["B03"]) / (
        df["B02"] + df["B03"]
    )
    # Calculate NDWI
    # (N - S1) / (N + S1)
    df["NDWI"] = (df["B02"] - df["B01"]) / (
        df["B02"] + df["B01"]
    )
    # Calculate G-NDVI
    # (N - G) / (N + G)
    df["G-NDVI"] = (df["B02"] - df["B04"]) / (
        df["B02"] + df["B04"]
    )
    # Calculate GRVI
    # (N / G)
    df["GRVI"] = df["B02"] / df["B04"]
    # # TGI	Triangular Greenness Index	-(-0.5*((R-B)*(R-G)-(R-G)*(R-B)))
    # df["TGI"] = -(-0.5 * (
    #     (df["B03"] - df["B05"]) * (df["B03"] - df["B04"])
    #     -
    #     (df["B03"] - df["B04"]) * (df["B03"] - df["B05"])
    #     )
    # )
    # TVI	Triangular Vegetation Index	0.5 * (120 * (N - G) - 200 * (R - G))
    df["TVI"] = 0.5 * (
        120 * (df["B02"] - df["B04"])
        - 200 * (df["B03"] - df["B04"])
    )

    # new_df = calculate_bands(df)

    # Get predicted probabilities
    test_preds = loaded_model.predict_proba(df.fillna(0))

    # Convert predicted probabilities to class predictions
    threshold = 0.9  # Adjust threshold as needed
    test_pred_labels = (test_preds[:, 1] > threshold).astype(int)

    # Plant predictions on the zero image
    zero_image[non_land_indices] = test_pred_labels

    # Generate the binary result
    binary_result = np.where(zero_image > 0, 1, 0)

    # Save res
    # Write the binary image to a GeoTIFF file
    output_binary_image = f"D:\\projects\\kelp_wanted\\data\\test_tiff_data\\{file.split('_')[0]}_kelp.tif"
    # output_binary_image = (
    #     r"D:\projects\kelp_wanted\data\test_tiff_data\JS569579_kelp_90.tif"
    # )

    with rasterio.open(
        output_binary_image,
        "w",
        driver="GTiff",
        height=height,
        width=width,
        count=1,  # Single band
        dtype=binary_result.dtype,
        crs=src.crs,
        # transform=src.transform,
    ) as dst:
        # Write the final binary image to the GeoTIFF file
        dst.write(binary_result, 1)

In [ ]:
# Read the id_kelp.tif file
with rasterio.open(test_file) as kelp_src:
    kelp_mask = kelp_src.read(1)
    band_data = kelp_src.read().astype(np.float32)  # Convert to float32

# Create a zero image of the same dimension
zero_image = np.zeros_like(kelp_mask)

# """
# Read all bands
# Get the shape of the image
height, width = kelp_src.height, kelp_src.width
# Apply scale factor and offset to all bands except band 6 and band 7
for i in range(5):
    band_data[i] = band_data[i] * scale_factor + offset

# Create a land mask
land_mask = band_data[6] > 0  # Assuming band 6 represents land
# """
# Read non-land pixels from the input image
non_land_pixels = band_data[:, ~land_mask]

# Remember the positions of non-land pixels
non_land_indices = np.where(~land_mask)

# Extract band values for each selected pixel
pixel_values = band_data[:, non_land_indices[0], non_land_indices[1]].T

# Create a DataFrame for the selected pixels
df = pd.DataFrame(pixel_values, columns=[f"B0{i+1}" for i in range(7)])

# dfs.append(df)

# # Concatenate all DataFrames in the list
# test_final_df = pd.concat(df, ignore_index=True)

# # Shuffle the DataFrame rows
# test_df_shuffled = test_final_df.sample(frac=1, random_state=42).reset_index(drop=True)

c:\Users\HDSL77\anaconda3\envs\bd_maskrcnn\lib\site-packages\rasterio\__init__.py:331: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


In [ ]:
df

In [ ]:
# # Calculate NDVI
# # (N - R)/(N + R)
# df["NDVI"] = (df["B02"] - df["B03"]) / (
#     df["B02"] + df["B03"]
# )
# # Calculate NDWI
# # (N - S1) / (N + S1)
# df["NDWI"] = (df["B02"] - df["B01"]) / (
#     df["B02"] + df["B01"]
# )
# # Calculate G-NDVI
# # (N - G) / (N + G)
# df["G-NDVI"] = (df["B02"] - df["B04"]) / (
#     df["B02"] + df["B04"]
# )
# # Calculate GRVI
# # (N / G)
# df["GRVI"] = df["B02"] / df["B04"]
# # # TGI	Triangular Greenness Index	-(-0.5*((R-B)*(R-G)-(R-G)*(R-B)))
# # df["TGI"] = -(-0.5 * (
# #     (df["B03"] - df["B05"]) * (df["B03"] - df["B04"])
# #     -
# #     (df["B03"] - df["B04"]) * (df["B03"] - df["B05"])
# #     )
# # )
# # TVI	Triangular Vegetation Index	0.5 * (120 * (N - G) - 200 * (R - G))
# df["TVI"] = 0.5 * (
#     120 * (df["B02"] - df["B04"])
#     - 200 * (df["B03"] - df["B04"])
# )

In [ ]:
df

In [ ]:
# # Get predicted probabilities
# test_preds = loaded_model.predict_proba(df.fillna(0))

# # Convert predicted probabilities to class predictions
# threshold = 0.9  # Adjust threshold as needed
# test_pred_labels = (test_preds[:, 1] > threshold).astype(int)

In [ ]:
(np.where(test_pred_labels == 0))[0].shape

(106291,)

In [ ]:
# # Plant predictions on the zero image
# zero_image[non_land_indices] = test_pred_labels

# # Generate the binary result
# binary_result = np.where(zero_image > 0, 1, 0)

In [ ]:
binary_result.shape

(350, 350)

In [ ]:
# Write the binary image to a GeoTIFF file
output_binary_image = r"D:\projects\kelp_wanted\data\test_data\JS569579_kelp_90.tif"

# # Calculate the new transformation matrix with a vertical flip
# transform = Affine(
#     test_src.transform.a,
#     0,
#     test_src.transform.c,
#     0,
#     -test_src.transform.e,
#     test_src.transform.f,
# )

with rasterio.open(
    output_binary_image,
    "w",
    driver="GTiff",
    height=height,
    width=width,
    count=1,  # Single band
    dtype=binary_result.dtype,
    crs=src.crs,
    # transform=src.transform,
) as dst:
    # Write the final binary image to the GeoTIFF file
    dst.write(binary_result, 1)

In [ ]:
# # Read the GeoTIFF file
# with rasterio.open(test_file) as test_src:
#     # Read all bands
#     band_data = test_src.read().astype(np.float32)  # Convert to float32
#     # Get the shape of the image
#     height, width = test_src.height, test_src.width
#     # Apply scale factor and offset to all bands except band 6 and band 7
#     for i in range(5):
#         band_data[i] = band_data[i] * scale_factor + offset

#     # Reshape each band's data to 1-dimensional array
#     reshaped_bands = {f"B0{i+1}": band_data[i].ravel() for i in range(5)}

#     # Create a DataFrame from the dictionary
#     test_df = pd.DataFrame(reshaped_bands)

In [ ]:
# # Get predicted probabilities
# test_preds = model.predict_proba(test_df.fillna(0))

In [ ]:
# # Convert predicted probabilities to class predictions
# threshold = 0.5  # Adjust threshold as needed
# test_pred_labels = (test_preds[:, 1] > threshold).astype(int)

In [ ]:
# import rasterio
# from rasterio.transform import from_origin
# from rasterio.enums import Resampling
# from rasterio.warp import calculate_default_transform, reproject, Resampling

# # Define output file path for the binary image
# # output_binary_image = "output_binary_image.tif"
# output_binary_image = (
#     r"D:\projects\kelp_wanted\data\test_data\AF169120_res.tif"
# )

# # Get spatial properties of the original GeoTIFF file
# transform = test_src.transform
# crs = test_src.crs
# dtype = test_src.dtypes[0]

# # Define pixel values for the two classes (0 and 1)
# class_0_value = 0
# class_1_value = 255  # Assuming 8-bit integer image

# # Reshape the predicted labels to match the original image dimensions
# test_pred_labels_reshaped = test_pred_labels.reshape(height, width)

# # Calculate the new transformation matrix with a vertical flip
# transform = Affine(
#     test_src.transform.a, 0, test_src.transform.c, 0, -test_src.transform.e, test_src.transform.f
# )

# # Write the binary image
# with rasterio.open(
#     output_binary_image,
#     "w",
#     driver="GTiff",
#     height=height,
#     width=width,
#     count=1,  # Single band
#     dtype=dtype,
#     crs=crs,
#     transform=transform,
# ) as dst:
#     # Write predicted labels to the binary image


#     dst.write(test_pred_labels_reshaped.astype(dtype), 1)

In [ ]:
## wrong code here!!!


import rasterio
import numpy as np
import pandas as pd

# Read the GeoTIFF file
with rasterio.open(test_file) as src:
    # Read all bands
    band_data = src.read().astype(np.float32)  # Convert to float32
    # Get the shape of the image
    height, width = src.height, src.width
    # Apply scale factor and offset to all bands except band 6 and band 7
    for i in range(5):
        band_data[i] = band_data[i] * scale_factor + offset

    # Create a land mask
    land_mask = band_data[6] > 0  # Assuming band 6 represents land
    # Apply the land mask to the test data
    test_data_non_land = band_data.copy()  # Make a copy of the original test data
    test_data_non_land[:, land_mask] = 0  # Set land pixels to 0 in all bands

    # Reshape test data to 2D array for prediction
    test_features = test_data_non_land.reshape(-1, 7)

# Make predictions using your model
test_preds = model.predict(test_features)

# Generate the binary image using the predicted labels
binary_image = np.zeros_like(
    land_mask, dtype=np.uint8
)  # Initialize binary image with zeros

# Reshape test_preds to match the shape of the boolean mask
test_preds_reshaped = test_preds.reshape(height, width)

# Assign predicted labels for non-land pixels
binary_image[~land_mask] = test_preds_reshaped[~land_mask]

# Write the binary image to a GeoTIFF file
output_binary_image = r"D:\projects\kelp_wanted\data\test_data\AF169120_res_2.tif"

# Calculate the new transformation matrix with a vertical flip
transform = Affine(
    test_src.transform.a,
    0,
    test_src.transform.c,
    0,
    -test_src.transform.e,
    test_src.transform.f,
)

with rasterio.open(
    output_binary_image,
    "w",
    driver="GTiff",
    height=height,
    width=width,
    count=1,  # Single band
    dtype=binary_image.dtype,
    crs=src.crs,
    transform=src.transform,
) as dst:
    # Write the final binary image to the GeoTIFF file
    dst.write(binary_image, 1)

In [ ]:
# Convert predicted probabilities to class predictions
threshold = 0.5  # Adjust threshold as needed
pred_labels = (preds[:, 1] > threshold).astype(int)

# Compute confusion matrix
conf_matrix = confusion_matrix(y_val, pred_labels)

# Compute total number of samples
total_samples = len(y_val)

# Convert confusion matrix to percentage values
conf_matrix_percentage = (conf_matrix / total_samples) * 100

print("Confusion Matrix (Percentage):")
print(conf_matrix_percentage)

In [ ]:
# Compute accuracy
accuracy = accuracy_score(y_val, pred_labels)

# Compute precision
precision = precision_score(y_val, pred_labels)

# Compute recall
recall = recall_score(y_val, pred_labels)

# Compute F1-score
f1 = f1_score(y_val, pred_labels)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

In [ ]:
# Generate confusion matrix
conf_matrix = confusion_matrix(y_val, pred_labels)

# Plot heatmap
plt.figure(figsize=(5, 5))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicted labels")
plt.ylabel("True labels")
plt.title("Confusion Matrix")
plt.show()

# Single Image test

In [21]:
test_file = r"D:\projects\kelp_wanted\data\train_satellite\JS569579_satellite.tif"

# Read the id_kelp.tif file
with rasterio.open(test_file) as kelp_src:
    kelp_mask = kelp_src.read(1)
    band_data = kelp_src.read().astype(np.float32)  # Convert to float32

# Create a zero image of the same dimension
zero_image = np.zeros_like(kelp_mask)

# """
# Read all bands
# Get the shape of the image
height, width = kelp_src.height, kelp_src.width
# Apply scale factor and offset to all bands except band 6 and band 7
for i in range(5):
    band_data[i] = band_data[i] * scale_factor + offset

# Create a land mask
land_mask = band_data[6] > 0  # Assuming band 6 represents land
# """
# Read non-land pixels from the input image
non_land_pixels = band_data[:, ~land_mask]

# Remember the positions of non-land pixels
non_land_indices = np.where(~land_mask)

# Extract band values for each selected pixel
pixel_values = band_data[:, non_land_indices[0], non_land_indices[1]].T

# Create a DataFrame for the selected pixels
df = pd.DataFrame(pixel_values, columns=[f"B0{i+1}" for i in range(7)])

# dfs.append(df)

# # Concatenate all DataFrames in the list
# test_final_df = pd.concat(df, ignore_index=True)

# # Shuffle the DataFrame rows
# test_df_shuffled = test_final_df.sample(frac=1, random_state=42).reset_index(drop=True)
# df
# Calculate NDVI
# (N - R)/(N + R)
df["NDVI"] = (df["B02"] - df["B03"]) / (df["B02"] + df["B03"])
# Calculate NDWI
# (N - S1) / (N + S1)
df["NDWI"] = (df["B02"] - df["B01"]) / (df["B02"] + df["B01"])
# Calculate G-NDVI
# (N - G) / (N + G)
df["G-NDVI"] = (df["B02"] - df["B04"]) / (df["B02"] + df["B04"])
# Calculate GRVI
# (N / G)
df["GRVI"] = df["B02"] / df["B04"]
# # TGI	Triangular Greenness Index	-(-0.5*((R-B)*(R-G)-(R-G)*(R-B)))
# df["TGI"] = -(-0.5 * (
#     (df["B03"] - df["B05"]) * (df["B03"] - df["B04"])
#     -
#     (df["B03"] - df["B04"]) * (df["B03"] - df["B05"])
#     )
# )
# TVI	Triangular Vegetation Index	0.5 * (120 * (N - G) - 200 * (R - G))
df["TVI"] = 0.5 * (120 * (df["B02"] - df["B04"]) - 200 * (df["B03"] - df["B04"]))


X = df.drop(columns=["B06", "B07"])

# Convert the Pandas DataFrame to a DMatrix
dtest = xgb.DMatrix(X)

# Use the loaded model to make predictions
test_preds = xgb_model.predict(dtest)

c:\Users\HDSL77\anaconda3\envs\bd_maskrcnn\lib\site-packages\rasterio\__init__.py:331: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import random
from tqdm import tqdm as tqdm
import pickle


# -------------------------------------------------
import lightgbm as lgbm
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from catboost import Pool, CatBoostClassifier
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GroupShuffleSplit

# ---------------------------------------------------
from sklearn.cluster import KMeans

# from featexp import get_univariate_plots
from imblearn.under_sampling import ClusterCentroids
from imblearn.under_sampling import TomekLinks
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer

# -------------------------data-----------------------#
import requests
from urllib.parse import urlparse
from pathlib import Path
from datetime import datetime

# Required libraries
import tifffile as tiff
import datetime
import matplotlib.pyplot as plt

import rasterio
from rasterio.plot import show
from rasterio.windows import Window
from rasterio.enums import Resampling
from rasterio.transform import Affine

import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Path to the input GeoTIFF file
input_file = r"D:\projects\kelp_wanted\data\train_satellite\AA498489_satellite.tif"
mask_file = r"D:\projects\kelp_wanted\data\train_kelp\AA498489_kelp.tif"

In [ ]:
train_satellite_path = r"D:\projects\kelp_wanted\data\train_satellite"
train_label_path = r"D:\projects\kelp_wanted\data\train_kelp"

In [ ]:
# Get the list of train files
train_input = os.listdir(train_satellite_path)
train_input_id = [d.split("_")[0] for d in train_input if d.endswith(".tif")]

# Get the list of train labels
train_label = os.listdir(train_label_path)

In [ ]:
train_input_id

In [ ]:
# Define scale factor and offset
scale_factor = 0.0000275
offset = -0.2

In [ ]:
# # make a dataframe with column for 7 bands, label and id
# df = pd.DataFrame(columns=["B01", "B02", "B03", "B04", "B05", "B06", "B07", "label", "id"])

In [ ]:
# train_input_id[:10]

In [ ]:
dfs = []  # Initialize list to store DataFrames
# count = 0

# Loop over each file
for file in train_input_id:  # Adjust the range as needed
    # Read the GeoTIFF file
    with rasterio.open(train_satellite_path + "\\" + file + "_satellite.tif") as src:
        # Read all bands
        band_data = src.read().astype(np.float32)  # Convert to float32
        # Get the shape of the image
        height, width = src.height, src.width
        # Apply scale factor and offset to all bands except band 6 and band 7
        for i in range(7):
            if i != 5 and i != 6:  # Skip band 6 and band 7
                band_data[i] = band_data[i] * scale_factor + offset

        # Create a mask where band 7 values are greater than 0
        land_mask = band_data[6] > 0
        # Apply the land mask to all bands except band 6 and 7
        for i in range(5):
            band_data[i][land_mask] = 0

        # Find the indices of pixels not in the land mask
        non_land_mask_indices = np.where(~land_mask)

        # Read the binary mask TIFF file
        with rasterio.open(train_label_path + "\\" + file + "_kelp.tif") as mask_src:
            # Read the mask data
            kelp_mask = mask_src.read(1)
            # Find the locations where the mask values are 1
            kelp_mask_indices = np.where(kelp_mask == 1)

        # Select a random sample of pixels
        num_random_samples = len(kelp_mask_indices[0])

        random_indices = random.sample(
            range(len(non_land_mask_indices[0])), num_random_samples
        )
        # Combine indices of kelp mask pixels and random sample pixels
        selected_indices = (
            np.concatenate(
                (kelp_mask_indices[0], non_land_mask_indices[0][random_indices])
            ),
            np.concatenate(
                (kelp_mask_indices[1], non_land_mask_indices[1][random_indices])
            ),
        )

        # Filter pixels with band 6 values greater than 0
        selected_indices_filtered = (
            selected_indices[0][band_data[5][selected_indices] == 0],
            selected_indices[1][band_data[5][selected_indices] == 0],
        )

        # Extract band values for each selected pixel
        pixel_values = band_data[
            :, selected_indices_filtered[0], selected_indices_filtered[1]
        ].T

        # Create a DataFrame for the selected pixels
        df = pd.DataFrame(pixel_values, columns=[f"B0{i+1}" for i in range(7)])

        # Create an array of labels
        labels = np.zeros(len(selected_indices_filtered[0]))
        labels[: len(kelp_mask_indices[0])] = 1  # Set labels to 1 for kelp mask pixels
        df["label"] = labels
        df["id"] = file
        # count = count + 1
        # df["count"] = count

        # Append DataFrame to the list
        dfs.append(df)

# Concatenate all DataFrames in the list
final_df = pd.concat(dfs, ignore_index=True)

# Shuffle the DataFrame rows
df_shuffled = final_df.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
len(np.where(df_shuffled.label == 0)[0])

4423238

In [ ]:
# # Read the GeoTIFF file
# with rasterio.open(input_file) as src:
#     # Read all bands
#     band_data = src.read().astype(np.float32)  # Convert to float32
#     # Get the shape of the image
#     height, width = src.height, src.width
#     # Apply scale factor and offset to all bands except band 6 and band 7
#     for i in range(7):
#         if i != 5 and i != 6:  # Skip band 6 and band 7
#             band_data[i] = band_data[i] * scale_factor + offset
            
#     # Create a mask where band 6 values are greater than 0
#     land_mask = band_data[6] > 0
#     # Apply the land mask to all bands except band 6
#     for i in range(7):
#         band_data[i][land_mask] = 0

#     # Find the indices of pixels not in the land mask
#     non_land_mask_indices = np.where(~land_mask)
    
#     # Read the binary mask TIFF file
#     with rasterio.open(mask_file) as mask_src:
#         # Read the mask data
#         kelp_mask = mask_src.read(1)
#         # Find the locations where the mask values are 1
#         kelp_mask_indices = np.where(kelp_mask == 1)

#     # Select a random sample of pixels
#     num_random_samples = min(
#         len(non_land_mask_indices[0]), len(kelp_mask_indices[0]), 1000
#     )
#     random_indices = random.sample(
#         range(len(non_land_mask_indices[0])), num_random_samples
#     )
#     # Combine indices of kelp mask pixels and random sample pixels
#     selected_indices = (
#         np.concatenate(
#             (kelp_mask_indices[0], non_land_mask_indices[0][random_indices])
#         ),
#         np.concatenate(
#             (kelp_mask_indices[1], non_land_mask_indices[1][random_indices])
#         ),
#     )

#     # Create a dictionary to store band values for each selected pixel
#     data_dict = {f"B0{i+1}": band_data[i][selected_indices] for i in range(7)}
#     # Create an array of labels
#     labels = np.zeros(len(selected_indices[0]))
#     labels[:len(kelp_mask_indices[0])] = 1  # Set labels to 1 for kelp mask pixels
    
#     # Add labels as a new column to the DataFrame
#     data_dict["label"] = labels
#     data_dict["id"] = input_id

#     # Create a DataFrame from the dictionary
#     df = pd.DataFrame(data_dict)
#     # Shuffle the DataFrame rows
#     df_shuffled = df.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
# Calculate NDVI
# (N - R)/(N + R)	
df_shuffled["NDVI"] = (df_shuffled["B02"] - df_shuffled["B03"]) / (
    df_shuffled["B02"] + df_shuffled["B03"]
)

In [ ]:
# Calculate NDWI
# (N - S1) / (N + S1)
df_shuffled["NDWI"] = (df_shuffled["B02"] - df_shuffled["B01"]) / (
    df_shuffled["B02"] + df_shuffled["B01"]
)

In [ ]:
# Calculate G-NDVI
# (N - G) / (N + G)
df_shuffled["G-NDVI"] = (df_shuffled["B02"] - df_shuffled["B04"]) / (
    df_shuffled["B02"] + df_shuffled["B04"]
)

In [ ]:
# Calculate GRVI 
# (N / G)
df_shuffled["GRVI"] = (df_shuffled["B02"] / df_shuffled["B04"])

In [ ]:
# # TGI	Triangular Greenness Index	-(-0.5*((R-B)*(R-G)-(R-G)*(R-B)))
# df_shuffled["TGI"] = -(-0.5 * (
#     (df_shuffled["B03"] - df_shuffled["B05"]) * (df_shuffled["B03"] - df_shuffled["B04"]) 
#     - 
#     (df_shuffled["B03"] - df_shuffled["B04"]) * (df_shuffled["B03"] - df_shuffled["B05"])
#     )
# )

In [ ]:
# TVI	Triangular Vegetation Index	0.5 * (120 * (N - G) - 200 * (R - G))
df_shuffled["TVI"] = 0.5 * (120 * (df_shuffled["B02"] - df_shuffled["B04"]) - 200 * (df_shuffled["B03"] - df_shuffled["B04"]))

In [ ]:
# # Calculate NIRv
# # ((N - R) / (N + R)) * N
# df_shuffled["NIRv"] = ((df_shuffled["B02"] - df_shuffled["B03"]) / (df_shuffled["B02"] + df_shuffled["B03"])) * df_shuffled["B03"]

In [ ]:
# # CIgreen(Chlorophyll Index Green)
# # (N / G) - 1.0
# df_shuffled["CIgreen"] = (df_shuffled["B02"] / df_shuffled["B04"])

In [ ]:
# # Find count of label 1 and 0
#     # label_counts = df_shuffled['label'].value_counts()


#     # Path to save the filtered GeoTIFF file
#     output_file = (
#         r"D:\projects\kelp_wanted\data\test_data\AA498489_satellite_filtered.tif"
#     )

#     # Calculate the new transformation matrix with a vertical flip
#     transform = Affine(
#         src.transform.a, 0, src.transform.c, 0, -src.transform.e, src.transform.f
#     )

#     # Write the filtered image to a new GeoTIFF file
#     with rasterio.open(
#         output_file,
#         "w",
#         driver="GTiff",
#         width=width,
#         height=height,
#         count=7,
#         dtype=band_data.dtype,
#         crs=src.crs,
#         transform=transform,
#     ) as dst:
#         for i in range(7):
#             dst.write(band_data[i], i + 1)

# print("Filtered image saved as:", output_file)

### feature importance after training
### Feature Importance: [ 7.9042736  24.65610458  8.17403439  4.31374823 10.15926229  7.50094776 11.60824219 10.62018195  6.10334926  8.95985576]

In [ ]:
df_shuffled.head(10)

In [ ]:
df_shuffled.to_csv(r"D:\projects\kelp_wanted\data\csv\df_shuffled.csv", index=False)

# predictions on the whole image

In [31]:
band_data.shape

(7, 350, 350)

In [ ]:
import rasterio

# Get predictions from XGBoost model (using entire image data)
test_preds = xgb_model.predict(dtest)

# Reshape predictions to match image dimensions
predicted_mask = test_preds.reshape(
    test_satellite_image.shape[0], test_satellite_image.shape[1]
)

# Create empty image for predicted mask
predicted_image = np.zeros_like(test_satellite_image, dtype=np.uint8)

# Handle land pixels (choose one option)
# Option 1: Assign specific value (e.g., 0)
predicted_image[land_mask] = 0

# Option 2: Use additional information (replace with your logic)
# if additional_info_indicates_non_kelp_on_land:
#     predicted_image[land_mask] = 0
# else:
#     # Handle land pixels based on your knowledge

# Assign predictions to non-land pixels
predicted_image[~land_mask] = predicted_mask[~land_mask]

# Save the predicted mask as a binary image
with rasterio.open(
    "predicted_kelp_mask.tif",
    "w",
    driver="GTiff",
    height=predicted_image.shape[0],
    width=predicted_image.shape[1],
    count=1,
    dtype=predicted_image.dtype,
) as dst:
    dst.write(predicted_image, 1)

In [28]:
type(test_preds)

numpy.ndarray

In [29]:
import numpy as np
import rasterio
from rasterio.transform import from_origin

# Assuming `test_preds` contains the predictions obtained from XGBoost

# Reshape the predictions array to match the shape of the satellite image
predicted_image = np.zeros_like(kelp_mask)
predicted_image[non_land_indices] = test_preds

# Generate the binary result
binary_result = np.where(predicted_image > 0.5, 1, 0)  # Adjust threshold as needed

# Write the binary image to a GeoTIFF file
output_binary_image = (
    r"D:\projects\kelp_wanted\data\test_data\JS569579_kelp_predicted.tif"
)

# Get metadata from the original satellite image
metadata = kelp_src.meta

# Update metadata for the output image
metadata.update(dtype=rasterio.uint8, count=1)

# Write the binary image to the GeoTIFF file
with rasterio.open(output_binary_image, "w", **metadata) as dst:
    dst.write(binary_result.astype(rasterio.uint8), 1)

c:\Users\HDSL77\anaconda3\envs\bd_maskrcnn\lib\site-packages\rasterio\__init__.py:341: NotGeoreferencedWarning: The given matrix is equal to Affine.identity or its flipped counterpart. GDAL may ignore this matrix and save no geotransform without raising an error. This behavior is somewhat driver-specific.
  dataset = writer(


In [ ]:
# Convert predicted probabilities to class predictions
threshold = 0.5  # Adjust threshold as needed
test_pred_labels = (test_preds[:, 1] > threshold).astype(int)
(np.where(test_pred_labels == 1))[0].shape
# Plant predictions on the zero image
zero_image[non_land_indices] = test_pred_labels

# Generate the binary result
binary_result = np.where(zero_image > 0, 1, 0)
binary_result.shape
# Write the binary image to a GeoTIFF file
output_binary_image = r"D:\projects\kelp_wanted\data\test_data\JS569579_kelp_50_xgb.tif"

# # Calculate the new transformation matrix with a vertical flip
# transform = Affine(
#     test_src.transform.a,
#     0,
#     test_src.transform.c,
#     0,
#     -test_src.transform.e,
#     test_src.transform.f,
# )

with rasterio.open(
    output_binary_image,
    "w",
    driver="GTiff",
    height=height,
    width=width,
    count=1,  # Single band
    dtype=binary_result.dtype,
    crs=src.crs,
    # transform=src.transform,
) as dst:
    # Write the final binary image to the GeoTIFF file
    dst.write(binary_result, 1)